In [53]:
import json
import re
from collections import defaultdict
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import accuracy_score
import os
import time

# Настройка путей
marks_dir = "301"  # Папка с JSON-файлами (marks/<id>.json)
combined_results_dir = "combined_results"  # Папка с текстами (<id>_combined.txt)
output_model_dir = "models"  # Папка для сохранения модели
scored_results_dir = "roberta_scored_results"  # Папка для файлов с оценками
os.makedirs(output_model_dir, exist_ok=True)
os.makedirs(scored_results_dir, exist_ok=True)

# Проверка существования папок
print(f"Папка marks существует: {Path(marks_dir).exists()}")
print(f"Папка combined_results существует: {Path(combined_results_dir).exists()}")

# Установка устройства (MPS для M3, иначе CPU)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Используемое устройство: {device}")

Папка marks существует: True
Папка combined_results существует: True
Используемое устройство: mps


In [54]:
# Функции для обработки данных
def parse_mask(mask):
    """
    Разбирает mask, например, "1(3)0(3)0(3)0(3)0(3)0(3)0(3)" → [1, 0, 0, 0, 0, 0, 0].
    """
    scores = re.findall(r'(\d)\(\d\)', mask)
    if len(scores) != 7:
        raise ValueError(f"Некорректный mask: {mask}, ожидается 7 оценок")
    return [int(score) for score in scores]

def read_combined_texts(file_path):
    """
    Читает файл <id>_combined.txt и возвращает словарь {task_num: текст}.
    """
    task_texts = defaultdict(list)
    current_task = None
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            for line in lines:
                line = line.strip()
                if not line:
                    continue
                if line.endswith(':'):
                    current_task = line[:-1]
                elif current_task:
                    task_texts[current_task].append(line)
    except Exception as e:
        print(f"Ошибка чтения файла {file_path}: {e}")
        return {}
    return {task: " ".join(texts) for task, texts in task_texts.items()}

# Класс для создания датасета
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        if len(texts) != len(labels):
            raise ValueError(f"Несоответствие длины: {len(texts)} текстов, {len(labels)} меток")
        print(f"Создан датасет: {len(texts)} элементов")

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Сбор данных
texts = []  # Список текстов (обучающая выборка)
labels = []  # Список оценок (обучающая выборка)
test_texts = []  # Список текстов (тестовая выборка)
test_labels = []  # Список оценок (тестовая выборка)
test_file_mapping = []  # Список пар (file_path, task_num, текст) для тестовых файлов
task_numbers = [str(i) for i in range(22, 29)]  # Задания 22–28

In [55]:
cnst = 370
# Собираем все файлы
all_files = sorted(Path(combined_results_dir).glob("*_combined.txt"))  # Сортируем для воспроизводимости
print(f"Найдено файлов в {combined_results_dir}: {len(all_files)}")
if len(all_files) < cnst:
    raise ValueError(f"Недостаточно файлов: найдено {len(all_files)}, требуется минимум {cnst}")

train_files = all_files[:cnst]
test_files = all_files[cnst:]

# Обучающая выборка
for combined_file in train_files:
    try:
        work_id = combined_file.stem.replace("_combined", "")
        json_file = Path(marks_dir) / f"{work_id}.json"
        if not json_file.exists():
            print(f"JSON-файл {json_file} не найден, пропуск {combined_file}")
            continue
        
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        mask = data.get("mask")
        if not mask:
            print(f"Пропуск {json_file}: отсутствует mask")
            continue
        
        try:
            scores = parse_mask(mask)
        except ValueError as e:
            print(f"Ошибка в маске {json_file}: {e}")
            continue
        
        task_texts = read_combined_texts(combined_file)
        if not task_texts:
            print(f"Пропуск {combined_file}: нет текстов")
            continue
        
        for i, task_num in enumerate(task_numbers):
            text = task_texts.get(task_num, "")
            if text:
                texts.append(text)
                labels.append(scores[i])
                print(f"Добавлен текст для {combined_file}, задание {task_num}, метка {scores[i]}")
            else:
                print(f"Пустой текст для {combined_file}, задание {task_num}")
    except Exception as e:
        print(f"Ошибка обработки {combined_file}: {e}")
        continue

# Тестовая выборка
for combined_file in test_files:
    try:
        work_id = combined_file.stem.replace("_combined", "")
        json_file = Path(marks_dir) / f"{work_id}.json"
        if not json_file.exists():
            print(f"JSON-файл {json_file} не найден, пропуск {combined_file}")
            continue
        
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        mask = data.get("mask")
        if not mask:
            print(f"Пропуск {json_file}: отсутствует mask")
            continue
        
        try:
            scores = parse_mask(mask)
        except ValueError as e:
            print(f"Ошибка в маске {json_file}: {e}")
            continue
        
        task_texts = read_combined_texts(combined_file)
        if not task_texts:
            print(f"Пропуск {combined_file}: нет текстов")
            continue
        
        for i, task_num in enumerate(task_numbers):
            text = task_texts.get(task_num, "")
            if text:
                test_texts.append(text)
                test_labels.append(scores[i])
                test_file_mapping.append((combined_file, task_num, text))
                print(f"Добавлен тестовый текст для {combined_file}, задание {task_num}, метка {scores[i]}")
            else:
                print(f"Пустой тестовый текст для {combined_file}, задание {task_num}")
    except Exception as e:
        print(f"Ошибка обработки {combined_file}: {e}")
        continue

# Проверка собранных данных
if not texts:
    raise ValueError("Нет данных для обучения: проверьте папки marks и combined_results")
if len(texts) != len(labels):
    raise ValueError(f"Несоответствие: {len(texts)} текстов, {len(labels)} меток")
print(f"Обучающая выборка: {len(texts)} текстов, {len(labels)} меток")
print(f"Тестовая выборка: {len(test_texts)} текстов, {len(test_labels)} меток")

Найдено файлов в combined_results: 419
Добавлен текст для combined_results/1019641987_combined.txt, задание 22, метка 1
Добавлен текст для combined_results/1019641987_combined.txt, задание 23, метка 1
Добавлен текст для combined_results/1019641987_combined.txt, задание 24, метка 1
Добавлен текст для combined_results/1019641987_combined.txt, задание 25, метка 1
Пустой текст для combined_results/1019641987_combined.txt, задание 26
Добавлен текст для combined_results/1019641987_combined.txt, задание 27, метка 2
Пустой текст для combined_results/1019641987_combined.txt, задание 28
Добавлен текст для combined_results/1020679752_combined.txt, задание 22, метка 0
Добавлен текст для combined_results/1020679752_combined.txt, задание 23, метка 1
Добавлен текст для combined_results/1020679752_combined.txt, задание 24, метка 0
Пустой текст для combined_results/1020679752_combined.txt, задание 25
Пустой текст для combined_results/1020679752_combined.txt, задание 26
Добавлен текст для combined_resul

In [56]:
# Инициализация токенизатора и модели RoBERTa
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=4)  # 4 класса: 0, 1, 2, 3
model.to(device)

# Подготовка датасетов
train_dataset = TextDataset(texts, labels, tokenizer)
test_dataset = TextDataset(test_texts, test_labels, tokenizer) if test_texts else None

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8) if test_dataset else None

# Обучение модели
optimizer = AdamW(model.parameters(), lr=2e-5)
model.train()
num_epochs = 3

start_time = time.time()
for epoch in range(num_epochs):
    total_loss = 0
    for i, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 50 == 0:
            print(f"Эпоха {epoch + 1}, батч {i}/{len(train_loader)}, время: {(time.time() - start_time) / 60:.2f} минут")
    
    avg_loss = total_loss / len(train_loader)
    print(f"Эпоха {epoch + 1}/{num_epochs}, Средняя потеря: {avg_loss:.4f}, Время: {(time.time() - start_time) / 60:.2f} минут")

# Сохранение модели
model.save_pretrained(Path(output_model_dir) / "roberta_classifier")
tokenizer.save_pretrained(Path(output_model_dir) / "roberta_classifier")
print("Модель RoBERTa сохранена в", output_model_dir)

# Проверка модели на тестовой выборке
if test_loader:
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Точность на тестовой выборке: {accuracy:.4f}")
else:
    print("Тестовая выборка пуста, пропуск проверки")

# Создание файлов с оценками для тестовых файлов
test_file_groups = defaultdict(list)
for file_path, task_num, text in test_file_mapping:
    test_file_groups[file_path].append((task_num, text))

model.eval()
with torch.no_grad():
    for combined_file, tasks in test_file_groups.items():
        try:
            output_lines = []
            for task_num, text in sorted(tasks):  # Сортируем по номеру задания
                encoding = tokenizer(
                    text,
                    add_special_tokens=True,
                    max_length=512,
                    padding='max_length',
                    truncation=True,
                    return_tensors='pt'
                )
                input_ids = encoding['input_ids'].to(device)
                attention_mask = encoding['attention_mask'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                score = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]
                output_lines.append(f"{task_num}: {score}\n{text}")
            
            output_file = Path(scored_results_dir) / combined_file.name
            with open(output_file, 'w', encoding='utf-8') as f:
                for line in output_lines:
                    f.write(line + "\n")
            print(f"Файл {output_file} создан с оценками")
        except Exception as e:
            print(f"Ошибка создания файла для {combined_file}: {e}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Создан датасет: 1748 элементов
Создан датасет: 235 элементов
Эпоха 1, батч 0/219, время: 0.11 минут
Эпоха 1, батч 50/219, время: 2.81 минут
Эпоха 1, батч 100/219, время: 6.25 минут
Эпоха 1, батч 150/219, время: 9.95 минут
Эпоха 1, батч 200/219, время: 13.94 минут
Эпоха 1/3, Средняя потеря: 1.2764, Время: 15.58 минут
Эпоха 2, батч 0/219, время: 15.72 минут
Эпоха 2, батч 50/219, время: 23.10 минут
Эпоха 2, батч 100/219, время: 32.02 минут
Эпоха 2, батч 150/219, время: 40.46 минут
Эпоха 2, батч 200/219, время: 57.87 минут
Эпоха 2/3, Средняя потеря: 1.2620, Время: 59.90 минут
Эпоха 3, батч 0/219, время: 60.07 минут
Эпоха 3, батч 50/219, время: 67.31 минут
Эпоха 3, батч 100/219, время: 74.38 минут
Эпоха 3, батч 150/219, время: 82.02 минут
Эпоха 3, батч 200/219, время: 91.12 минут
Эпоха 3/3, Средняя потеря: 1.2641, Время: 93.37 минут
Модель RoBERTa сохранена в models
Точность на тестовой выборке: 0.4809
Файл roberta_scored_results/2820712761_combined.txt создан с оценками
Файл roberta_scored

In [ ]:
Точность на тестовой выборке: 0.4809

{0: '22', 1: '23', 2: '24', 3: '25', 4: '26', 5: '27', 6: '28', 7: 'номер', 8: 'текст'}
